In [1]:
import pyspark as ps
from collections import Counter
import pandas as pd
import time
import matplotlib.pyplot as plt
spark = (ps.sql.SparkSession.builder 
        .master("local[4]") 
        .appName("nathanscope") 
        .getOrCreate()
        )

In [2]:
sc = spark.sparkContext

In [3]:
col_names = ['marketplace','customer_id','review_id','product_id','product_parent','product_title','product_category','star_rating','helpful_votes','total_votes','vine','verified_purchase','review_headline','review_body','review_date']
cols = {}
for i in range(len(col_names)):
    print (str(i)+': '+col_names[i])
    cols[col_names[i]] = i 

0: marketplace
1: customer_id
2: review_id
3: product_id
4: product_parent
5: product_title
6: product_category
7: star_rating
8: helpful_votes
9: total_votes
10: vine
11: verified_purchase
12: review_headline
13: review_body
14: review_date


In [4]:
def casting_function(row):
    if row[0] == 'marketplace':
        return ()
    if len(row)!=15:
        return(len(row))
    (marketplace, customer_id, review_id, product_id, product_parent, product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date) = row
    
    
    return (marketplace, str(customer_id), str(review_id), str(product_id), product_parent, str(product_title),str(product_category),int(star_rating),int(helpful_votes),int(total_votes),vine,verified_purchase,str(review_headline),str(review_body),review_date)




In [5]:
def time_cast(row):
    pattern = '%Y-%m-%d'
    row =list(row)
    row[14] = int(time.mktime(time.strptime(row[14], pattern)))/86400
    return tuple(row)

In [6]:
def toCSVLine(data):
  return ','.join(str(d) for d in data)

In [7]:
rdd_books = (sc.textFile('file:///Users/nathan/Desktop/Amazon/Amazon_Books/data/amazon_reviews_us_Books_v1_00.tsv.gz')
            .map(lambda rowstr : rowstr.split("\t"))
            .map(casting_function)
           )

In [8]:
rdd_books =rdd_books.filter(lambda x: len(x)==15).map(time_cast)

In [9]:
#rdd_sample = rdd_books.filter(lambda x: x[14]%50==0)
rdd_sample = rdd_books.filter(lambda x: x[7]==1)

star_counts = rdd_books.map(lambda x: Counter({x[cols['star_rating']]:1})).reduce(lambda x, y: x+y)

rdd_sample.count()

In [10]:
tiny_df = pd.DataFrame(rdd_sample.take(10))
tiny_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,US,34058159,R10QIRBIDAU34P,191052428X,942262122,The New Reaction,Books,1,9,12,N,N,Completely Terrible,"Fortunately, I read this for free via Amazon's...",16678.291667
1,US,9124640,R1YVO0YNQYFTY4,1566845181,878587221,The Baby Grand: A True Mother's Day Love Story,Books,1,5,5,N,Y,"This is an essay, not a short story, barely 17...","Unfortunately, I didn't realize what I was buy...",16678.291667
2,US,21366438,R28BDX8MNMRK0C,1464109273,99307772,Exploring Psychology in Modules (Loose Leaf),Books,1,0,0,N,Y,no.,zero stars if I could. I received the wrong book.,16678.291667
3,US,38851864,R3E7686UOSWVI7,0143039989,514925319,The Haunting of Hill House (Penguin Classics),Books,1,2,12,N,N,"What an underdeveloped, unoriginal waste of time","First off, I have no idea who is writing the 5...",16678.291667
4,US,52534000,R107KYALGWON5I,1629983004,944142850,El misterio del Shemitá: 3000 años de antigüed...,Books,1,0,0,N,N,Caution: Cahn's theory is contradicted by empi...,I am a lecturer in empirical research methods ...,16678.291667
5,US,4610105,R1O55T7M4CHTD0,1934759295,609341244,Buddha's Wife,Books,1,0,0,N,Y,While I normally love reading fictional storie...,While I normally love reading fictional storie...,16678.291667
6,US,8391381,R29E1HKT45SQ97,161767737X,561937478,Panorama 4th Supersite PLUS CODE (includes Cod...,Books,1,0,0,N,Y,TOOK MY MONEY/SENT ME NOTHING,this place is not legit,16678.291667
7,US,15960945,R2TTB52MHCBXZM,0764210661,935522247,Your Beautiful Purpose: Discovering and Enjoyi...,Books,1,0,2,N,Y,One Star,boring book.,16678.291667
8,US,15960945,RAU08QQ9G9TUF,B00CC6ETU2,486809418,"Finding the Game: Three Years, Twenty-five Cou...",Books,1,0,1,N,Y,but this book is horrifically boring.,I had to read this for college english and met...,16678.291667
9,US,34942316,RVZ6EWMAO9XY6,149703518X,483616363,Studyguide for Accounting Information Systems ...,Books,1,0,0,N,Y,Rip-off,This &#34;study guide&#34; basically takes ran...,16678.291667


In [11]:
#star_counts = rdd_sample.map(lambda x: Counter({x[cols['star_rating']]:1})).reduce(lambda x, y: x+y)

In [12]:
#rdd_sample.saveAsTextFile('file:///home/ec2-user/Amazon_Books/data/sample_02.tsv')

In [ ]:
temp = pd.DataFrame(rdd_sample.collect())

sum(['x8b'in x for x in temp[14]])

temp.to_csv('cows.csv')

a = u'\xa1'
unicode(a, "utf-8")

a.decode('utf-8', 'ignore')

In [ ]:
str('cows').encode('utf8')

In [ ]:
temp.to_csv('data/books_sample_1star.csv',index=False)

In [ ]:
new_sample = pd.read_csv('data/books_sample_1star.csv)

In [ ]:
helpful_percentage = []
for pos, total in zip(new_sample['8'],new_sample['9']):
    if total>0:
        helpful_percentage.append(float(pos)/float(total))
    else:
        helpful_percentage.append(0)

In [ ]:
new_sample['15'] = helpful_percentage

In [ ]:
help_df = new_sample[new_sample['8']>0]

In [ ]:
help_df

In [ ]:
plt.hist(help_df['15'])

In [ ]:
#help_df = help_df.sample(frac=.5, random_state=1)

In [ ]:
plt.scatter(help_df['15'],help_df['7'])

In [ ]:
#plt.hist(help_df[help_df['7']==1]['15'],normed=True,alpha=.5,color='red')
#plt.hist(help_df[help_df['7']==2]['15'],normed=True,alpha=.5,color='orange')
#plt.hist(help_df[help_df['7']==3]['15'],normed=True,alpha=.5,color='yellow')
plt.hist(help_df[help_df['7']==4]['15'],normed=True,alpha=.5,color='blue')
plt.hist(help_df[help_df['7']==5]['15'],normed=True,alpha=.5,color='green')

In [ ]:
sum(help_df['15']<=.8)

In [ ]:
plt.scatter(help_df['15'],help_df['9'])

In [ ]:
plt.scatter(help_df['15'],help_df['9'])
plt.ylim(0,200)

In [ ]:
help_df[help_df['9']>0]['15'].mean()

In [ ]:
help_df[help_df['9']>15]['15'].mean()

In [ ]:
help_df['9'].median()

In [ ]:
plt.hist(help_df['9'],bins=100)
plt.xlim(0,100)

In [ ]:
len(help_df[help_df['9']>3])

In [ ]:
sum(help_df['15']>=.66)

In [ ]:
sum(help_df['15']<.66)

In [ ]:
good_df = help_df[help_df['15']>=.66]
bad_df = help_df[help_df['15']<.66]